# Pandas Tutorial

following this realpython tutorial: https://realpython.com/python-pandas-tricks/

## 1. Configure Pandas Settings on Startup

In [1]:
import pandas as pd

def start():
    options = {
        'display': {
            'max_columns': None,
            'max_colwidth': 25,
            'expand_frame_repr': False,  # Don't wrap to multiple pages
            'max_rows': 14,
            'max_seq_items': 50,         # Max length of printed sequence
            'precision': 4,
            'show_dimensions': False
        },
        'mode': {
            'chained_assignment': None   # Controls SettingWithCopyWarning
        }
    }

    for category, option in options.items():
        for op, value in option.items():
            pd.set_option(f'{category}.{op}', value)  # Python 3.6+

if __name__ == '__main__':
    start()
    del start  # Clean up namespace in the interpreter
    print(pd.__name__)
    print(pd.get_option('display.max_rows'))


pandas
14


get some data to demonstrate the settings:

In [2]:
url = ('https://archive.ics.uci.edu/ml/'
       'machine-learning-databases/abalone/abalone.data')
cols = ['sex', 'length', 'diam', 'height', 'weight', 'rings']
abalone = pd.read_csv(url, usecols=[0, 1, 2, 3, 4, 8], names=cols)

In [3]:
abalone

,sex,length,diam,height,weight,rings
0,M,0.455,0.365,0.095,0.5140,15
1,M,0.350,0.265,0.090,0.2255,7
2,F,0.530,0.420,0.135,0.6770,9
3,M,0.440,0.365,0.125,0.5160,10
4,I,0.330,0.255,0.080,0.2050,7
5,I,0.425,0.300,0.095,0.3515,8
6,F,0.530,0.415,0.150,0.7775,20
...,...,...,...,...,...,...
4170,M,0.550,0.430,0.130,0.8395,10
4171,M,0.560,0.430,0.155,0.8675,8


here we see the 14 rows and 4 digits we set earlier

## 2. Make Toy Data Structures With Pandas’ Testing Module

Hidden way down in Pandas’ `testing` module are a number of convenient functions for quickly building quasi-realistic Series and DataFrames:

In [4]:
import pandas.util.testing as tm
tm.N, tm.K = 15, 3  # Module-level default rows/columns

tm.N


15

In [5]:
import numpy as np
np.random.seed(42)

tm.makeTimeDataFrame(freq='M').head()

,A,B,C
2000-01-31,0.4967,-0.5623,-0.6017
2000-02-29,-0.1383,-1.0128,1.8523
2000-03-31,0.6477,0.3142,-0.0135
2000-04-30,1.5230,-0.9080,-1.0577
2000-05-31,-0.2342,-1.4123,0.8225


In [6]:
tm.makeDataFrame().head()

,A,B,C
0xzy777SOC,-0.7198,0.7673,-0.3655
oSaygixaRh,-0.0186,-1.1498,0.6491
xkYqhIIG6e,-1.6735,-0.7753,-1.2229
PM5OBgihlH,-1.0725,0.7731,0.5363
GV2w9xKIRN,-0.9926,-0.8018,-0.9147


In [7]:
[i for i in dir(tm) if i.startswith('make')]

['makeBoolIndex',
 'makeCategoricalIndex',
 'makeCustomDataframe',
 'makeCustomIndex',
 'makeDataFrame',
 'makeDateIndex',
 'makeFloatIndex',
 'makeFloatSeries',
 'makeIntIndex',
 'makeIntervalIndex',
 'makeMissingCustomDataframe',
 'makeMissingDataframe',
 'makeMixedDataFrame',
 'makeMultiIndex',
 'makeObjectSeries',
 'makePanel',
 'makePeriodFrame',
 'makePeriodIndex',
 'makePeriodPanel',
 'makePeriodSeries',
 'makeRangeIndex',
 'makeStringIndex',
 'makeStringSeries',
 'makeTimeDataFrame',
 'makeTimeSeries',
 'makeTimedeltaIndex',
 'makeUIntIndex',
 'makeUnicodeIndex']

In [8]:
tm.makeFloatSeries().head()

baV9leKF62    0.9051
iOIsVpctx1   -0.6039
3GxZkWhJLN    0.3044
t9IV9yIyCr    0.2572
Trb1Ip8OJG    0.0239
dtype: float64

These can be useful for benchmarking, testing assertions, and experimenting with Pandas methods that you are less familiar with.

## 3. Take Advantage of Accessor Methods

In [9]:
pd.Series._accessors

{'cat', 'dt', 'str'}

.cat is for categorical data, .str is for string (object) data, and .dt is for datetime-like data. Let’s start off with .str: imagine that you have some raw city/state/ZIP data as a single field within a Pandas Series.

In [10]:
addr = pd.Series([
    'Washington, D.C. 20003',
    'Brooklyn, NY 11211-1755',
    'Omaha, NE 68154',
    'Pittsburgh, PA 15211'
])

addr.str.upper()


0     WASHINGTON, D.C. 20003
1    BROOKLYN, NY 11211-1755
2            OMAHA, NE 68154
3       PITTSBURGH, PA 15211
dtype: object

In [11]:
addr.str.count(r'\d')  # 5 or 9-digit zip?

0    5
1    9
2    5
3    5
dtype: int64

The second accessor, .dt, is for datetime-like data. It technically belongs to Pandas’ DatetimeIndex, and if called on a Series, it is converted to a DatetimeIndex first:

In [12]:
daterng = pd.Series(pd.date_range('2017', periods=9, freq='Q'))
daterng

0   2017-03-31
1   2017-06-30
2   2017-09-30
3   2017-12-31
4   2018-03-31
5   2018-06-30
6   2018-09-30
7   2018-12-31
8   2019-03-31
dtype: datetime64[ns]

In [13]:
daterng.dt.day_name()

0      Friday
1      Friday
2    Saturday
3      Sunday
4    Saturday
5    Saturday
6      Sunday
7      Monday
8      Sunday
dtype: object

In [14]:
# Second-half of year only
daterng[daterng.dt.quarter > 2]

2   2017-09-30
3   2017-12-31
6   2018-09-30
7   2018-12-31
dtype: datetime64[ns]

In [15]:
daterng[daterng.dt.is_year_end]

3   2017-12-31
7   2018-12-31
dtype: datetime64[ns]

## 4. Create a DatetimeIndex From Component Columns

Speaking of datetime-like data, as in daterng above, it’s possible to create a Pandas DatetimeIndex from multiple component columns that together form a date or datetime:

In [16]:
from itertools import product
datecols = ['year', 'month', 'day']

df = pd.DataFrame(list(product([2017, 2016], [1, 2], [1, 2, 3])),
                  columns=datecols)
df['data'] = np.random.randn(len(df))
df

,year,month,day,data
0,2017,1,1,-0.8559
1,2017,1,2,-0.0396
2,2017,1,3,-0.5347
3,2017,2,1,-1.7885
4,2017,2,2,0.3573
5,2017,2,3,-0.4145
6,2016,1,1,0.0802
7,2016,1,2,-0.8931
8,2016,1,3,-0.3348
9,2016,2,1,1.5604


creating an index from multiple column names


In [17]:
df.index = pd.to_datetime(df[datecols])
df

,year,month,day,data
2017-01-01,2017,1,1,-0.8559
2017-01-02,2017,1,2,-0.0396
2017-01-03,2017,1,3,-0.5347
2017-02-01,2017,2,1,-1.7885
2017-02-02,2017,2,2,0.3573
2017-02-03,2017,2,3,-0.4145
2016-01-01,2016,1,1,0.0802
2016-01-02,2016,1,2,-0.8931
2016-01-03,2016,1,3,-0.3348
2016-02-01,2016,2,1,1.5604


Finally, you can drop the old individual columns and convert to a Series:

In [18]:
df = df.drop(datecols, axis=1).squeeze()
df.head()

2017-01-01   -0.8559
2017-01-02   -0.0396
2017-01-03   -0.5347
2017-02-01   -1.7885
2017-02-02    0.3573
Name: data, dtype: float64

In [19]:
df.index.dtype_str

'datetime64[ns]'

## 5. Use Categorical Data to Save on Time and Space

In [20]:
colors = pd.Series([
    'periwinkle',
    'mint green',
    'burnt orange',
    'periwinkle',
    'burnt orange',
    'rose',
    'rose',
    'mint green',
    'rose',
    'navy'
])

In [21]:
import sys
colors.apply(sys.getsizeof)

0    59
1    59
2    61
3    59
4    61
5    53
6    53
7    59
8    53
9    53
dtype: int64

In [23]:
colors.memory_usage()

160

Now, what if we could take the unique colors above and map each to a less space-hogging integer? Here is a naive implementation of that:

In [24]:
mapper = {v: k for k, v in enumerate(colors.unique())}
mapper

{'periwinkle': 0, 'mint green': 1, 'burnt orange': 2, 'rose': 3, 'navy': 4}

In [25]:
as_int = colors.map(mapper)
as_int

0    0
1    1
2    2
3    0
4    2
5    3
6    3
7    1
8    3
9    4
dtype: int64

In [26]:
as_int.apply(sys.getsizeof)

0    24
1    28
2    28
3    24
4    28
5    28
6    28
7    28
8    28
9    28
dtype: int64

In [27]:
sys.getsizeof(as_int)

184

In [29]:
as_int.memory_usage()

160

Note: Another way to do this same thing is with Pandas’ pd.factorize(colors):

In [30]:
pd.factorize(colors)[0]

array([0, 1, 2, 0, 2, 3, 3, 1, 3, 4])

more information and more detail in the above tutorial link.

## 6. Introspect Groupby Objects via Iteration

When you call df.groupby('x'), the resulting Pandas groupby objects can be a bit opaque. This object is lazily instantiated and doesn’t have any meaningful representation on its own.

In [33]:
abalone.head()

,sex,length,diam,height,weight,rings,ring_quartile
0,M,0.455,0.365,0.095,0.5140,15,4
1,M,0.350,0.265,0.090,0.2255,7,1
2,F,0.530,0.420,0.135,0.6770,9,2
3,M,0.440,0.365,0.125,0.5160,10,3
4,I,0.330,0.255,0.080,0.2050,7,1


In [31]:
abalone['ring_quartile'] = pd.qcut(abalone.rings, q=4, labels=range(1, 5))
grouped = abalone.groupby('ring_quartile')

grouped

In [32]:
grouped.head()

,sex,length,diam,height,weight,rings,ring_quartile
0,M,0.455,0.365,0.095,0.5140,15,4
1,M,0.350,0.265,0.090,0.2255,7,1
2,F,0.530,0.420,0.135,0.6770,9,2
3,M,0.440,0.365,0.125,0.5160,10,3
4,I,0.330,0.255,0.080,0.2050,7,1
5,I,0.425,0.300,0.095,0.3515,8,1
6,F,0.530,0.415,0.150,0.7775,20,4
...,...,...,...,...,...,...,...
13,F,0.535,0.405,0.145,0.6845,10,3
14,F,0.470,0.355,0.100,0.4755,10,3


Alright, now you have a groupby object, but what is this thing, and how do I see it?

Before you call something like grouped.apply(func), you can take advantage of the fact that groupby objects are iterable:

In [34]:
help(grouped.__iter__)

Help on method __iter__ in module pandas.core.groupby.groupby:

__iter__() method of pandas.core.groupby.groupby.DataFrameGroupBy instance
    Groupby iterator
    
    Returns
    -------
    Generator yielding sequence of (name, subsetted object)
    for each group



In [35]:
for idx, frame in grouped:
    print(f'Ring quartile: {idx}')
    print('-' * 16)
    print(frame.nlargest(3, 'weight'), end='\n\n')

Ring quartile: 1
----------------
     sex  length   diam  height  weight  rings ring_quartile
2619   M   0.690  0.540   0.185  1.7100      8             1
1044   M   0.690  0.525   0.175  1.7005      8             1
1026   M   0.645  0.520   0.175  1.5610      8             1

Ring quartile: 2
----------------
     sex  length  diam  height  weight  rings ring_quartile
2811   M   0.725  0.57   0.190  2.3305      9             2
1426   F   0.745  0.57   0.215  2.2500      9             2
1821   F   0.720  0.55   0.195  2.0730      9             2

Ring quartile: 3
----------------
     sex  length  diam  height  weight  rings ring_quartile
1209   F   0.780  0.63   0.215   2.657     11             3
1051   F   0.735  0.60   0.220   2.555     11             3
3715   M   0.780  0.60   0.210   2.548     11             3

Ring quartile: 4
----------------
     sex  length   diam  height  weight  rings ring_quartile
891    M   0.730  0.595    0.23  2.8255     17             4
1763   M   0.77

In [36]:
grouped.groups.keys()

dict_keys([1, 2, 3, 4])

In [41]:
grouped.get_group(3).head()

,sex,length,diam,height,weight,rings,ring_quartile
3,M,0.440,0.365,0.125,0.5160,10,3
11,M,0.430,0.350,0.110,0.4060,10,3
12,M,0.490,0.380,0.135,0.5415,11,3
13,F,0.535,0.405,0.145,0.6845,10,3
14,F,0.470,0.355,0.100,0.4755,10,3


In [42]:
grouped['height', 'weight'].agg(['mean', 'median'])

height         weight        
                 mean median    mean  median
ring_quartile                               
1              0.1066  0.105  0.4324  0.3685
2              0.1427  0.145  0.8520  0.8440
3              0.1572  0.155  1.0669  1.0645
4              0.1648  0.165  1.1149  1.0655

## 7. Use This Mapping Trick for Membership Binning

Let’s say that you have a Series and a corresponding “mapping table” where each value belongs to a multi-member group, or to no groups at all:

In [43]:
countries = pd.Series([
    'United States',
    'Canada',
    'Mexico',
    'Belgium',
    'United Kingdom',
    'Thailand'
])

In [44]:
groups = {
    'North America': ('United States', 'Canada', 'Mexico', 'Greenland'),
    'Europe': ('France', 'Germany', 'United Kingdom', 'Belgium')
}

In [46]:
from typing import Any

def membership_map(s: pd.Series, groups: dict,
                   fillvalue: Any=-1) -> pd.Series:
    # Reverse & expand the dictionary key-value pairs
    groups = {x: k for k, v in groups.items() for x in v}
    return s.map(groups).fillna(fillvalue)

In [47]:
membership_map(countries, groups, fillvalue='other')

0    North America
1    North America
2    North America
3           Europe
4           Europe
5            other
dtype: object

In [48]:
groups = dict(enumerate(('ab', 'cd', 'xyz')))
{x: k for k, v in groups.items() for x in v}

{'a': 0, 'b': 0, 'c': 1, 'd': 1, 'x': 2, 'y': 2, 'z': 2}

## 8. Understand How Pandas Uses Boolean Operators

In [49]:
# Evaluates to "False and True"
4 < 3 and 5 > 4

False

In [51]:
# Evaluates to 4 < 5 > 4
4 < (3 and 5) > 4

True

In [52]:
pd.Series([True, True, False]) & pd.Series([True, False, False])

0     True
1    False
2    False
dtype: bool

In [54]:
s = pd.Series(range(10))

In [55]:
s

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int64

In [56]:
s % 2 == 0 & s > 3

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

What’s happening here? It’s helpful to incrementally bind the expression with parentheses, spelling out how Python expands this expression step by step:

In [58]:
# s % 2 == 0 & s > 3                      # Same as above, original expression
(s % 2) == 0 & s > 3                    # Modulo is most tightly binding here
(s % 2) == (0 & s) > 3                  # Bitwise-and is second-most-binding
(s % 2) == (0 & s) and (0 & s) > 3      # Expand the statement
((s % 2) == (0 & s)) and ((0 & s) > 3)  # The `and` operator is least-binding

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

... see full tutorial.

## 9. Load Data From the Clipboard

a   b           c       d
0   1           inf     1/1/00
2   7.389056099 N/A     5-Jan-13
4   54.59815003 nan     7/24/18
6   403.4287935 None    NaT

Simply highlight and copy the plain text above, and call pd.read_clipboard():

In [60]:
df = pd.read_clipboard(na_values=[None], parse_dates=['d'])
df

PyperclipException: 
    Pyperclip could not find a copy/paste mechanism for your system.
    For more information, please visit https://pyperclip.readthedocs.org 

In [61]:
df.dtypes

dtype('float64')

## 10. Write Pandas Objects Directly to Compressed Format

In [62]:
abalone.to_json('df.json.gz', orient='records',
                lines=True, compression='gzip')